In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# SATELITE PREDICTION

#  LOADING DATA

In [1]:
from utils import *
import warnings

warnings.filterwarnings("ignore")

In [2]:
train_path = "train.csv"
test_path = "test.csv"
submission_path = "submission.csv"

In [3]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
submission = pd.read_csv(submission_path)

In [4]:
train["epoch"] = pd.to_datetime(train["epoch"])
test["epoch"] = pd.to_datetime(test["epoch"])

In [5]:
train.sort_values(["sat_id", "epoch"], inplace=True)
test.sort_values(["sat_id", "epoch"], inplace=True)

### We concatenate the train and the test to align the periods.

In [6]:
train["test"] = 0
test["test"] = 1
tmp = pd.concat([train, test], ignore_index=True)

### We calculate the time interval between each satellite observation and we calculate the periods, we then drop the time intervals that are useless.

In [7]:
tmp = calcul_by_satellite(tmp, period_dt_calcul)
tmp.drop("dt", axis=1, inplace=True)

### Resplit  train and test

In [8]:
train = tmp.query("test == 0")
test = tmp.query("test == 1")

In [9]:
train.drop(["test"], inplace=True, axis=1)
test.drop(["test"], inplace=True, axis=1)

### Fill empty values (just in case)

In [10]:
train.fillna(method="bfill", inplace=True)
# test.fillna(method='bfill',inplace=True)

### We store the dataFrame in a temporary variable because we need to leave it intact to retrieve the val labels and the fit label.

In [11]:
current = train.copy()

# Validation process

### We separate our new train in two time intervals, be careful here we separate on current and not on the train (We want to preserve it!).

In [13]:
day = 15
train_before_day, train_after_day = separate(current, day)[0], separate(current, day)[1]

<IPython.core.display.Javascript object>

### We select only the columns that interest us for the fit and the val.

In [14]:


fit = train_before_day.drop(["id","x_sim","y_sim","z_sim","Vx_sim","Vy_sim","Vz_sim"],axis=1)

fit_label = separate(train,day)[0][["x","y","z","Vx","Vy","Vz","sat_id"]]


val = train_after_day.drop(["x_sim","y_sim","z_sim","Vx_sim","Vy_sim","Vz_sim","id"],axis=1)

val_label = separate(train,day)[1][["x","y","z","Vx","Vy","Vz",'sat_id','id']]

rest_pred = train_after_day[["id","sat_id"]]


<IPython.core.display.Javascript object>

## Compute on fit

In [15]:
fit = calcul_by_satellite(fit, cycle_calcul)

<IPython.core.display.Javascript object>

## We repeat the first period of the train in the fit

In [16]:
fit = repeat_first_per(train_before_day, fit)  # We can optimize this function

<IPython.core.display.Javascript object>

## Compute val cycle

In [23]:
val = calcul_by_satellite(val, cycle_calcul)

<IPython.core.display.Javascript object>

In [24]:
val

,Vx,Vy,Vz,sat_id,x,y,z,period,cycle
464,1.164458,-0.172971,2.691682,0,2547.770935,-39617.771874,6960.838287,7,0
465,1.098782,0.457837,2.522866,0,5729.546164,-39197.570080,14291.169898,8,0
466,0.996558,1.001706,2.272088,0,8673.544191,-37132.414730,21027.789795,9,0
467,0.866281,1.464646,1.958794,0,11290.138540,-33657.584731,26970.314515,10,0
468,0.713236,1.851735,1.595038,0,13508.709516,-28992.406198,31961.792120,11,0
...,...,...,...,...,...,...,...,...,...
649907,-1.604980,3.219404,0.090053,599,-20751.580366,-15862.254305,5185.802272,16,38
649908,-0.876992,3.663390,-0.084036,599,-22620.279726,-10717.768931,5192.966657,17,38
649909,0.006030,3.963108,-0.278490,599,-23288.937188,-5011.995253,4925.091680,18,38
649910,1.041712,4.050118,-0.488054,599,-22525.993612,992.105147,4355.082480,19,38


<IPython.core.display.Javascript object>

## We repeat the first period of the fit in the val

In [25]:
repeat_last_per(train_before_day, val)

<IPython.core.display.Javascript object>

In [26]:
val

,Vx,Vy,Vz,sat_id,x,y,z,period,cycle
464,1.164458,-0.172971,2.691682,0,2547.770935,-39617.771874,6960.838287,7,0
465,1.098782,0.457837,2.522866,0,5729.546164,-39197.570080,14291.169898,8,0
466,0.996558,1.001706,2.272088,0,8673.544191,-37132.414730,21027.789795,9,0
467,0.866281,1.464646,1.958794,0,11290.138540,-33657.584731,26970.314515,10,0
468,0.713236,1.851735,1.595038,0,13508.709516,-28992.406198,31961.792120,11,0
...,...,...,...,...,...,...,...,...,...
649907,-1.604980,3.219404,0.090053,599,-20751.580366,-15862.254305,5185.802272,16,38
649908,-0.876992,3.663390,-0.084036,599,-22620.279726,-10717.768931,5192.966657,17,38
649909,0.006030,3.963108,-0.278490,599,-23288.937188,-5011.995253,4925.091680,18,38
649910,1.041712,4.050118,-0.488054,599,-22525.993612,992.105147,4355.082480,19,38


<IPython.core.display.Javascript object>

## Add somme feature here if you want

In [27]:
# fit = feature_by_satellite(fit)
# fit = features_by_period(fit)
# val = feature_by_satellite(val)
# val = features_by_period(val)

<IPython.core.display.Javascript object>

In [28]:
prediction = separate(train, day)[1][["id", "sat_id"]]

<IPython.core.display.Javascript object>

### Polynomial regression

In [31]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)


dico = {
    "x": ["x", "Vx", "cycle"],
    "y": ["y", "Vy", "cycle"],
    "z": ["z", "Vz", "cycle"],
    "Vx": ["Vx", "x", "cycle"],
    "Vy": ["Vy", "y", "cycle"],
    "Vz": ["Vz", "z", "cycle"],
}
for sat in tqdm_notebook(fit["sat_id"].unique()):
    for target in ["x", "y", "z", "Vx", "Vy", "Vz"]:
        # train sur le satelite sat
        fit_sat = fit.loc[fit["sat_id"] == sat]
        fit_sat = fit_sat[dico[target]]
        fit_sat = poly.fit_transform(fit_sat)

        model = LinearRegression(n_jobs=-1)
        model.fit(fit_sat, fit_label.query("sat_id == @sat")[target])
        val_sat = val.loc[val["sat_id"] == sat]
        val_sat = val_sat[dico[target]]
        val_sat = poly.fit_transform(val_sat)

        prediction.loc[prediction["sat_id"] == sat, target] = model.predict(val_sat)

display_smape(prediction, val_label)


Score : 93.47968368515298


<IPython.core.display.Javascript object>

# Model evaluation on test set

In [12]:
current = repeat_first_per(train, current)
current = calcul_by_satellite(current, cycle_calcul)

## Compute cycle

In [13]:
test = calcul_by_satellite(test, cycle_calcul)

## The fit contains the train (where the first period was repeated) and the val contains the last period of the repeated train.

In [14]:


fit = current.drop(["id","x_sim","y_sim","z_sim","Vx_sim","Vy_sim","Vz_sim","epoch"],axis=1).copy()

fit_label = train[["x","y","z","Vx","Vy","Vz","sat_id"]]

repeat_last_per(train,test)
val = test
val.drop(["x_sim","y_sim","z_sim","Vx_sim","Vy_sim","Vz_sim"],axis=1,inplace=True)



## Prediction dataframe

In [15]:
prediction = test[["id", "sat_id"]]

In [16]:
val.drop(["id", "epoch"], inplace=True, axis=1)

In [21]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)

dico = {"x" : ["x","Vx","cycle"],
        "y" : ["y","Vy","cycle"],
        "z" : ["z","Vz","cycle"],
        "Vx" : ["Vx","x","cycle"],
        "Vy" : ["Vy","y","cycle"],
        "Vz" : ["Vz","z","cycle"]}

for sat in tqdm_notebook(val["sat_id"].unique()):
    for target in ["x","y","z","Vx","Vy","Vz"]:
        #train sur le satelite sat
        fit_sat = fit.loc[fit['sat_id'] == sat]
        fit_sat = fit_sat[dico[target]]
        fit_sat = poly.fit_transform(fit_sat)

        model = LinearRegression(n_jobs=-1)
        model.fit(fit_sat,fit_label.query('sat_id == @sat')[target])
        val_sat = val.loc[val['sat_id'] == sat]
        val_sat = val_sat[dico[target]]
        val_sat = poly.fit_transform(val_sat)

        prediction.loc[prediction['sat_id'] == sat,target] = model.predict(val_sat)


In [22]:
prediction.drop(["sat_id"], axis=1, inplace=True)

In [23]:
prediction

,id,x,y,z,Vx,Vy,Vz
649912,3927,-24151.505176,-10550.571617,5913.660135,2.378560,-1.367786,1.150305
649913,3928,-20626.476558,-11992.126719,7125.072025,3.017256,-1.090245,0.946616
649914,3929,-16355.753387,-13009.351505,8140.398973,3.742219,-0.689965,0.636661
649915,3930,-11264.756474,-13389.920840,8855.792920,4.552615,-0.071117,0.145050
649916,3931,-5285.528902,-12751.484795,9068.014769,5.368228,0.995465,-0.710844
...,...,...,...,...,...,...,...
933978,1231060,76689.645129,-59341.057426,-531.048166,-0.807003,-1.297647,0.160673
933979,1231061,69869.540010,-68172.752525,654.086544,-1.041348,-1.096156,0.160328
933980,1231062,61422.280987,-75424.479820,1825.011828,-1.250180,-0.870089,0.156271
933981,1231063,51529.213269,-80907.383721,2953.924131,-1.434354,-0.617185,0.148417


In [24]:
prediction.to_csv("pred2.csv",index=False)